In [1]:
!pip install aspect-based-sentiment-analysis
!pip install sentencepiece

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import keras
from keras import layers
from keras.layers import Input
from keras.models import Model


from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

import aspect_based_sentiment_analysis as absa

from textblob import TextBlob
from nltk.stem.porter import PorterStemmer



import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from textblob import TextBlob
from nltk.stem.porter import PorterStemmer



import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure required NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
# nltk.download("popular")

INFO: pip is looking at multiple versions of aspect-based-sentiment-analysis to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.3 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Data & Lexicon Preprocessing

In [3]:
yelp_reviews = pd.read_csv("/content/gdrive/MyDrive/SYSEN 6888/Code/yelp_dataset/yelp reviews.csv", encoding='latin1')

<ipython-input-3-a52ca60c1d14>:1: DtypeWarning: Columns (0,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  yelp_reviews = pd.read_csv("/content/gdrive/MyDrive/SYSEN 6888/Code/yelp_dataset/yelp reviews.csv", encoding='latin1')


In [4]:
yelp_reviews.shape

(1048575, 19)

In [5]:
# remove NaN reviews
yelp_reviews['Comment'].map(type).unique()
drop_indices = yelp_reviews[yelp_reviews['Comment'].map(type) != str].index.tolist()

yelp_reviews = yelp_reviews.drop(drop_indices)

In [6]:
yelp_reviews.shape

(1048574, 19)

In [7]:
yelp_reviews['Length'] = yelp_reviews['Comment'].apply(lambda x: len(x.split()))

In [8]:
yelp_reviews = yelp_reviews[['Length', 'Comment']]

In [9]:
# np.sort(yelp_reviews['Length'].unique())

In [10]:
yelp_reviews.shape

(1048574, 2)

In [11]:
# short reviews (words of length [5,40])
short_reviews = yelp_reviews[(yelp_reviews['Length'] > 5) & (yelp_reviews['Length'] <= 40)].reset_index()
print(short_reviews.shape)
short_reviews.head()

(239241, 3)


,index,Length,Comment
0,1,31,What would you expect from a cart? Lots of foo...
1,5,17,Ordered the Chirashi. It was average tasting. ...
2,25,35,Cool place if you happen to be on Washington. ...
3,26,24,Wings are the best in Hoboken. Reasonably pric...
4,40,17,Haven't had a bad experience yet. They have pr...


In [12]:
# long reviews (paragraph, [40,150])
long_reviews = yelp_reviews[(yelp_reviews['Length'] > 40) & (yelp_reviews['Length'] < 150)].reset_index()
print(long_reviews.shape)
long_reviews.head()

(580446, 3)


,index,Length,Comment
0,0,65,This street cart -- vaguely where 147 Centre S...
1,2,47,very bland rolls...no spice whatsoever in the ...
2,3,66,Can't say anything about their rolls because I...
3,4,106,I love this place. I'm so shocked that nobody ...
4,6,142,"Chimi chimi cocoa pop, chimi chimi rock... Sea..."


# ANALYSIS/COMPARISONS

In [ ]:
# import random
final = []
for i in range(5):

  final.append(random.randint(0,len(short_reviews)))

final

[72261, 88015, 220288, 104113, 47817]

short reviews indices: [7192,  48502,  52475,  80773,  85571,  95526, 126164, 158729, 189959, 192087, 193590, 200569, 203869]
long reviews indices: [ 44823,  62302,  82119, 183012, 317114, 320417, 324000, 335281, 341490, 404605, 457119, 478264, 488909, 532973]


CSV FILE with Columns: Index, Review, sentiment analysis output, your comments/analysis on 1) accuracy of the labels (pos, neutral, neg), 2) comprehensiveness/robustness of the aspects, 3) any comparisons to the other sentiment models
THEN COLUMN 4: final comparison/analysis

In [23]:
pyabsa_short = pd.read_csv('/content/gdrive/MyDrive/SYSEN 6888/Code/short_review_pyabsa.csv')
pyabsa_short.rename(columns={'Sentiment Analysis Output': 'PyABSA - Output', 'Comments/Analysis': 'PyABSA - Analysis'}, inplace=True)

pyabsa_short.head()

,Index,PyABSA - Output,PyABSA - Analysis
0,7192,"{'aspect': ['lunch buffet', 'food', 'variety',...",The aspects identified align with the sentimen...
1,48502,"{'aspect': ['atmosphere', 'staff', 'food', 'Eg...",The aspects identified align with the sentimen...
2,52475,"{'aspect': ['food', 'choices of food', 'rice']...",The model performed poorly for this review. Fa...
3,80773,"{'aspect': ['BBQ', 'delivery', 'food'], 'senti...","The positive sentiment on BBQ is accurate, the..."
4,85571,"{'aspect': ['tuna tartare', 'fried chicken', '...",Failed to capture all the food mentioned in th...


In [34]:
pyabsa_long = pd.read_csv('/content/gdrive/MyDrive/SYSEN 6888/Code/long_review_pyabsa.csv')
pyabsa_long.rename(columns={'Sentiment Analysis Output': 'PyABSA - Output', 'Comments/Analysis': 'PyABSA - Analysis'}, inplace=True)

pyabsa_long.head()

,Index,PyABSA - Output,PyABSA - Analysis
0,44823,"{'aspect': ['food', 'chicken tikka masala', 's...",The aspects identified align with the negative...
1,62302,"{'aspect': ['waitresses', 'atmosphere'], 'sent...",The model captures the positive sentiments abo...
2,82119,"{'aspect': ['staff', 'food', 'margarita pizza'...",The positive sentiments align well with the as...
3,183012,"{'aspect': ['menu', 'decor', 'service', 'waite...","Negative sentiments on menu, service, and food..."
4,317114,"{'aspect': ['Pricing', 'much'], 'sentiment': [...",The model captures the negative sentiment on p...


In [35]:
lexicon_absa_analysis_long = pd.DataFrame(columns=['Index', 'Length', 'Review', 'Lexicon ABSA - Output'])#, 'Lexicon ABSA - Analysis'])

# format output for comparison
def review_and_sentiment(df, indices, label):
  # 0 = short, 1 = long
  global lexicon_absa_analysis_long
  for index in indices:
    review = df.iloc[index]['Comment']
    aspect_sentiment = analyze_aspect_sentiment(review, lexicon_sets)

    lexicon_absa_analysis_long.loc[len(lexicon_absa_analysis_long.index)] = [index, label, review, aspect_sentiment]

    print(f'INDEX:{index}\nREVIEW: {review}\nSENTIMENT: {aspect_sentiment}\n')

In [36]:
long_indices = [44823,  62302,  82119, 183012, 317114, 320417, 324000, 335281, 341490, 404605, 457119, 478264, 488909, 532973]
review_and_sentiment(long_reviews, long_indices, 1)

INDEX:44823
REVIEW: I ordered food through Seamless. The food was horrendous. The chicken tikka masala was the worst: over buttered tasteless red sauce containing cubes of stale chicken. The lamb appetizer was inedible, which was very hard and fibrous. The rice lost all its softness and was dehydrated. Perhaps this was a one time experience. They don't get a second chance from me. Eat at your own risk.
SENTIMENT: {'ambience': [], 'cuisine': [], 'food': [('chicken tikka masala', 'Negative'), ('rice', 'Negative'), ('chicken', 'Negative'), ('lamb', 'Negative'), ('sauce', 'Negative'), ('butter', 'Negative')], 'price': [], 'restaurant': [], 'service': [], 'staff': []}

INDEX:62302
REVIEW: A diverse crowd comes here to dine because there is literally no one on earth who can dislike Bono. It is so delicious--I've had almost all of their pizzas which they take the meat out of for me as I am vegetarian. You can watch the chef cook your pizza in the oven since it is outside of the kitchen! My bo

In [37]:
lexicon_absa_analysis_long.head()

,Index,Length,Review,Lexicon ABSA - Output
0,44823,1,I ordered food through Seamless. The food was ...,"{'ambience': [], 'cuisine': [], 'food': [('chi..."
1,62302,1,A diverse crowd comes here to dine because the...,"{'ambience': [('atmosphere', 'Positive')], 'cu..."
2,82119,1,Simply a great place for dinner. Especially af...,"{'ambience': [], 'cuisine': [], 'food': [('piz..."
3,183012,1,This is the worst Cuban/Mexican restaurant I'v...,"{'ambience': [('decor', 'Positive')], 'cuisine..."
4,317114,1,Ok I've been back a few times since and can sa...,"{'ambience': [], 'cuisine': [], 'food': [('sau..."


In [38]:
final_df_long = pd.merge(lexicon_absa_analysis_long, pyabsa_long, on="Index")

In [39]:
final_df_long.to_csv('/content/gdrive/MyDrive/SYSEN 6888/Code/long_review_sentiment.csv')

In [27]:
short_indices = [7192,  48502,  52475,  80773,  85571,  95526, 126164, 158729, 189959, 192087, 193590, 200569, 203869]
review_and_sentiment(short_reviews, short_indices, 0)

INDEX:7192
REVIEW: Went there for lunch buffet. I always crave Indian food so loved it.. the variety is good from dosa to pani puri to North Indian to nice desserts. liked the experience :)
SENTIMENT: {'ambience': [], 'cuisine': [('indian', 'Positive')], 'food': [('dosa', 'Positive')], 'price': [], 'restaurant': [], 'service': [], 'staff': []}

INDEX:48502
REVIEW: Love this place! Great atmosphere, great staff and awesome food! I love the Eggs Benedict !
SENTIMENT: {'ambience': [('atmosphere', 'Positive')], 'cuisine': [], 'food': [('eggs benedict', 'Positive')], 'price': [], 'restaurant': [], 'service': [], 'staff': [('staff', 'Positive')]}

INDEX:52475
REVIEW: Awesome food market near Grand Central Park. The choices of food is excellent. I usually have either Korean food or any mixed (sushi, fish, salad, rice, tofu, noodle) food.
SENTIMENT: {'ambience': [], 'cuisine': [('korean', 'Positive')], 'food': [('tofu', 'Neutral'), ('rice', 'Neutral'), ('sushi', 'Neutral'), ('noodle', 'Neutral

In [31]:
final_df.head()

,Index,Length,Review,Lexicon ABSA - Output,PyABSA - Output,PyABSA - Analysis
0,7192,0,Went there for lunch buffet. I always crave In...,"{'ambience': [], 'cuisine': [('indian', 'Posit...","{'aspect': ['lunch buffet', 'food', 'variety',...",The aspects identified align with the sentimen...
1,48502,0,"Love this place! Great atmosphere, great staff...","{'ambience': [('atmosphere', 'Positive')], 'cu...","{'aspect': ['atmosphere', 'staff', 'food', 'Eg...",The aspects identified align with the sentimen...
2,52475,0,Awesome food market near Grand Central Park. T...,"{'ambience': [], 'cuisine': [('korean', 'Posit...","{'aspect': ['food', 'choices of food', 'rice']...",The model performed poorly for this review. Fa...
3,80773,0,"BBQ is very good, but as to delivery they left...","{'ambience': [], 'cuisine': [], 'food': [], 'p...","{'aspect': ['BBQ', 'delivery', 'food'], 'senti...","The positive sentiment on BBQ is accurate, the..."
4,85571,0,Good but not great by any means. Ordered 6-7 d...,"{'ambience': [], 'cuisine': [], 'food': [('fri...","{'aspect': ['tuna tartare', 'fried chicken', '...",Failed to capture all the food mentioned in th...


In [30]:
final_df = pd.merge(lexicon_absa_analysis, pyabsa_short, on="Index")

In [33]:
final_df.to_csv('/content/gdrive/MyDrive/SYSEN 6888/Code/short_review_sentiment.csv')

In [ ]:
# LONG
# ambience


# cuisine

# food

# price

# restaurant

# service

# staff



review_and_sentiment(long_reviews, [212123, 73004, 457606, 64068, 541412])


REVIEW: Great spot. Even better because of where it's located. Awesome grits. Awesome pickles. If you have a stroller, they'll lock it on the side which make things super easy for those of us with kids. Good sized portions, and yum, yum, yummy food.
SENTIMENT: {'ambience': [], 'cuisine': [], 'food': [], 'price': [], 'restaurant': [], 'service': [], 'staff': []}

REVIEW: Amazing amazing amazing. One of my favorite "fast food" joints in the city. Considering they only have three options on their menu (lobster, lobster roll and burger), it's not completely unwarranted that the chefs whip the meals out quickly. Not only is the service fast- the food is absolutely delicious. I've been here twice, and have ordered the lobster roll both times. After my first lobster roll experience, I couldn't resist ordering it again. However, friends did rave about the burger, as well. The cocktails are also very yummy- they have a pretty good selection of beers, wine and specialty cocktails. The interior i

# LEXICON PREPROCESSING

In [13]:
# Lexicon file names
lexicon_files = ["ambience", "cuisine", "food", "price", "restaurant", "service", "staff"]

# Load lexicons and convert to lowercase sets
lexicon_sets = {}
for name in lexicon_files:
    lexicon_df = pd.read_csv(f"/content/gdrive/MyDrive/SYSEN 6888/Code/yelp_lexicons/{name}.csv", header=None)
    lexicon_df[0] = lexicon_df[0].str.lower()  # Convert to lowercase
    lexicon_sets[name] = set(lexicon_df[0].values.ravel())

In [14]:
# Aspect Identification helper functions
# Source: https://borgesalkan.medium.com/yelp-popular-dishes-with-aspect-based-sentiment-analysis-796c191245bf
def _extract_ngrams(data, num):
    n_grams = TextBlob(data).ngrams(num)
    return [' '.join(grams).lower() for grams in n_grams]

def _delete_duplicate_ngrams(text, ngrams):
    ngrams.sort(key=lambda x: -len(x.split()))  # Sort desc by number of words
    result_ngrams = []
    for ngram in ngrams:
        if ngram in text:
            text = text.replace(ngram, '')
            result_ngrams.append(ngram)
    return result_ngrams

def extract_aspects(text, lexicons):
    text = text.lower()
    aspects = set()
    stemmer = PorterStemmer()
    for n in range(6, 0, -1):
        n_grams = _extract_ngrams(text, n)
        n_grams_stemmed = [stemmer.stem(n_gram) for n_gram in n_grams]
        n_grams_set = set(n_grams).union(n_grams_stemmed)
        aspects = aspects.union(n_grams_set.intersection(lexicons))
    return _delete_duplicate_ngrams(text, list(aspects))

In [15]:
# Load the ABSA model and tokenizer
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [16]:
# Analyze aspect sentiment and group by lexicon type
def analyze_aspect_sentiment(caption, lexicon_sets):
    final_sentiment = {lexicon: [] for lexicon in lexicon_sets}
    for lexicon, lexicon_set in lexicon_sets.items():
        aspects = extract_aspects(caption, lexicon_set)
        for aspect in aspects:
            sentiment_result = classifier(caption, text_pair=aspect)
            sentiment_label = sentiment_result[0]['label']
            final_sentiment[lexicon].append((aspect, sentiment_label))
    return final_sentiment

In [17]:
review_and_sentiment(short_reviews, [0,1])

REVIEW: What would you expect from a cart? Lots of food! For $1! I got the noodles of course. For a dollar it was good and greasy. If only she spoke english...
SENTIMENT: {'ambience': [], 'cuisine': [], 'food': [('noodles', 'Positive')], 'price': [], 'restaurant': [], 'service': [], 'staff': []}

REVIEW: Ordered the Chirashi. It was average tasting. It was not bad but nothing worth really writing about.
SENTIMENT: {'ambience': [], 'cuisine': [], 'food': [], 'price': [], 'restaurant': [], 'service': [], 'staff': []}



In [18]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## PyABSA

In [ ]:
# !pip install fastapi kaleido python-multipart uvicorn
# !pip install -U pyabsa

In [ ]:
# remove NaN reviews
yelp_reviews['Comment'].map(type).unique()
drop_indices = yelp_reviews[yelp_reviews['Comment'].map(type) != str].index.tolist()

yelp_reviews = yelp_reviews.drop(drop_indices)

new_review = df.iloc[index]['Comment']